# Proportional Approval Voting implementation

See also C code at [CleanOptPRVote.c](https://rangevoting.org/CleanOptPRVote.c) for a related problem as discussed by RangeVoting\.org at:
* [Optimal proportional representation Part I:  methods, algorithms, advantages, and inherent flaws](https://rangevoting.org/QualityMulti.html)
* [Optimal proportional representation Part II: The two classes and the "holy grail"](https://rangevoting.org/NonlinQuality.html)

Todo: When it gets slow
* use counts of unique cvrs, not all cvrs
* use multiple threads
* come up with more efficient algorithm, e.g. A*, which is easily distributed
* recode in Rust :)

In [1]:
import string
import pandas as pd
import collections
import itertools
import time

import pav

In [60]:
winner, w_cvrs, scores, df = pav.tally_pav("Westminster_Adams.csv", 3)

Plurality results in order
Anita_Seitz         4921
Shannon_Bird        4823
Maria_De_Cambra     3921
Mark_Clark          3500
David_DeMott        3126
Debbie_Bergamo      2668
Mike_Melvin         2112
Jason_Blanckaert    2026
Steve_Caulk         2026
Nathan_Pearce       1229
dtype: int64

12684 CVRs, 169 unique selections of candidates
Score is 10619.167 for plurality winners ['Anita_Seitz', 'Shannon_Bird', 'Maria_De_Cambra']

Proportional Approval Voting results

Top 10 scores:
('10799.500', ['Anita_Seitz', 'Mark_Clark', 'Shannon_Bird'])
('10619.167', ['Anita_Seitz', 'Maria_De_Cambra', 'Shannon_Bird'])
('10607.500', ['Anita_Seitz', 'Debbie_Bergamo', 'Shannon_Bird'])
('10588.833', ['Anita_Seitz', 'David_DeMott', 'Shannon_Bird'])
('10366.333', ['Maria_De_Cambra', 'Mark_Clark', 'Shannon_Bird'])
('10349.500', ['Anita_Seitz', 'Mike_Melvin', 'Shannon_Bird'])
('10260.667', ['Anita_Seitz', 'Jason_Blanckaert', 'Shannon_Bird'])
('10214.000', ['Anita_Seitz', 'Maria_De_Cambra', 'Mark_Clark'])
('

In [3]:
# Use this to test out new code after modifying the pav module
import importlib
importlib.reload(pav)

<module 'pav' from '/srv/s/approvalvoting/pr_voting_methods/pav.py'>

In [4]:
df.head()

,Mark_Clark,Mike_Melvin,Maria_De_Cambra,Shannon_Bird,Steve_Caulk,Debbie_Bergamo,Anita_Seitz,Jason_Blanckaert,David_DeMott,Nathan_Pearce
0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,1,1,0,0
4,0,0,0,1,0,1,0,0,1,0


In [61]:
winner, a_cvrs, scores, df = pav.tally_pav("Byers_SD_32J_Adams.csv", 4)

Plurality results in order
Tom_Thompson_III        81
Jerry_L_Sauer           75
Christopher_P_Cary      57
Julie_Smith             56
Donna_J_Sauer           54
Yvonne_M_Gerhardt       47
Jennifer_Simanovicki    36
Margaret_A_Holeman      23
dtype: int64

121 CVRs, 50 unique selections of candidates
Score is 182.333 for plurality winners ['Tom_Thompson_III', 'Jerry_L_Sauer', 'Christopher_P_Cary', 'Julie_Smith']

Proportional Approval Voting results

Top 10 scores:
('182.333', ['Donna_J_Sauer', 'Jerry_L_Sauer', 'Julie_Smith', 'Tom_Thompson_III'])
('182.333', ['Christopher_P_Cary', 'Jerry_L_Sauer', 'Julie_Smith', 'Tom_Thompson_III'])
('180.833', ['Jerry_L_Sauer', 'Julie_Smith', 'Tom_Thompson_III', 'Yvonne_M_Gerhardt'])
('179.583', ['Christopher_P_Cary', 'Jerry_L_Sauer', 'Tom_Thompson_III', 'Yvonne_M_Gerhardt'])
('179.333', ['Christopher_P_Cary', 'Donna_J_Sauer', 'Jerry_L_Sauer', 'Tom_Thompson_III'])
('177.583', ['Donna_J_Sauer', 'Jerry_L_Sauer', 'Tom_Thompson_III', 'Yvonne_M_Gerhardt'])

# Timing results

Note that the C code at [CleanOptPRVote.c](https://rangevoting.org/CleanOptPRVote.c) has been timed at 4V·Binomial(C,W) nanoseconds.

Based on the result below of 162 seconds for 24 candidates, 12 winners, 121 ballots,
this code might be perhaps 2 orders of magnitude slower, with a constant multiplier of 500 nanoseconds rather than 4.

Takes about 12 seconds(?) for 20 candidates, 10 winners, 121 ballots

Uses nearly 3 GB of memory with 24 and 12

24 choose 12 = 2704156

In [84]:
def timepav(candidates, winners, cvrs):
    "Time how long it takes to tally a PAV contest using given cvrs with given number of candidates and winners"

    # If you need more than 62 candidates for some unusual reason, see itertools.product or this infinite recipe: https://stackoverflow.com/a/42099359/507544
    many_candidates = tuple(string.ascii_uppercase + string.ascii_lowercase + string.digits)
    possible_results = itertools.combinations(many_candidates[:candidates], winners)
    t1 = time.time()
    scores = {}
    for result in possible_results:
        resultset = frozenset(result)
        scores[resultset] = sum(pav.utility(resultset, cvr) for cvr in cvrs)
    timings.append("{} seconds for {} candidates, {} winners, {} ballots".format(time.time() - t1, candidates, winners, len(cvrs)))

In [85]:
timings = []

In [86]:
timepav(12, 6, a_cvrs)

In [87]:
timepav(12, 6, w_cvrs)

In [89]:
timepav(14, 7, a_cvrs)

In [90]:
timepav(14, 7, w_cvrs)

In [92]:
timepav(18, 9, w_cvrs)

In [93]:
timings

['0.06837797164916992 seconds for 12 candidates, 6 winners, 121 ballots',
 '5.542601108551025 seconds for 12 candidates, 6 winners, 12684 ballots',
 '0.22739052772521973 seconds for 14 candidates, 7 winners, 121 ballots',
 '19.80630850791931 seconds for 14 candidates, 7 winners, 12684 ballots',
 '309.81311416625977 seconds for 18 candidates, 9 winners, 12684 ballots']

In [59]:
assert False, "Prevent overwriting of previous results below"

AssertionError: Prevent overwriting of previous results below

In [285]:
# older results
timings

['0.005549907684326172 seconds for 8 candidates, 4 winners, 121 ballots',
 '0.0004897117614746094 seconds for 8 candidates, 4 winners, 121 ballots',
 '0.022046566009521484 seconds for 10 candidates, 5 winners, 121 ballots',
 '0.22809982299804688 seconds for 14 candidates, 7 winners, 121 ballots',
 '2.5786209106445312 seconds for 18 candidates, 9 winners, 121 ballots',
 '9.772108793258667 seconds for 20 candidates, 10 winners, 121 ballots',
 '39.48445129394531 seconds for 22 candidates, 11 winners, 121 ballots',
 '161.45553970336914 seconds for 24 candidates, 12 winners, 121 ballots',
 '4.725412607192993 seconds for 24 candidates, 5 winners, 121 ballots',
 '6.555292844772339 seconds for 30 candidates, 5 winners, 121 ballots']

# Harmonic / Psi voting
From [Optimal proportional representation - RangeVoting\.org](https://rangevoting.org/QualityMulti.html)

Let C=#candidates, V=#voters, W=#winners, 0<W<C<V, and let S_v,c be the real-valued score given by voter v to candidate c on her ballot. Then here is our highly general class of quality measures Q:


Q = ∑1≤v≤V F( ∑1≤j≤W [jth greatest S_v,c among winning c] · A_j )

"Harmonic voting.") If F(x)=x, then PR will be assured by choosing A_j = 1/(j-1+Δ) where Δ is any positive constant.

I.e. for delta = 1/2, A_j = 2 * [1, 1/3, 1/5, 1/7, ...]

And for delta = 1, A_j = [1, 1/2, 1/3, 1/4, ...]

In [96]:
def A(j, delta=0.5):
    return 1 / (j - 1 + delta)

In [98]:
[A(j) for j in range(1, 5)]

[2.0, 0.6666666666666666, 0.4, 0.2857142857142857]

In [99]:
[A(j, 1.0) for j in range(1, 5)]

[1.0, 0.5, 0.3333333333333333, 0.25]

# Older

In [4]:
candidates = 12
winners = 6

In [45]:
many_candidates = tuple(string.ascii_uppercase + string.ascii_lowercase + string.digits)

In [36]:
possible_results = itertools.combinations(all_candidates + tuple(list(range(candidates - len(all_candidates)))), winners)

In [38]:
t1 = time.time()
scores = {}
for result in possible_results:
    resultset = frozenset(result)
    scores[resultset] = sum(pav.utility(resultset, cvr) for cvr in cvrs)
timings.append("{} seconds for {} candidates, {} winners, {} ballots".format(time.time() - t1, candidates, winners, len(cvrs)))

In [80]:
timings

['1031.4427270889282 seconds for 20 candidates, 10 winners, 121 ballots',
 '5.157426595687866 seconds for 12 candidates, 6 winners, 12684 ballots']